In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [2]:
data_df=pd.read_csv("data/diabetes.csv")
clean_data_df = data_df[
    (data_df['Glucose'] != 0) &
    (data_df['BloodPressure'] != 0) &
    (data_df['SkinThickness'] != 0) &
    (data_df['Insulin'] != 0) &
    (data_df['BMI'] != 0)
]

In [3]:
def glucoseTrap(glucose: ctrl.Antecedent):
    glucose['very low'] = fuzz.trapmf(glucose.universe, [0, 54, 69, 97.5])
    glucose['low'] = fuzz.trapmf(glucose.universe, [53, 71, 96.5, 125])
    glucose['medium'] = fuzz.trapmf(glucose.universe, [70, 98.5, 124, 162.5])
    glucose['high'] = fuzz.trapmf(glucose.universe, [97.5, 126, 161.5, 200])
    glucose['very high'] = fuzz.trapmf(glucose.universe, [125, 163.5, 199, 300])


def glucoseTri(glucose: ctrl.Antecedent):
    glucose['very low'] = fuzz.trapmf(glucose.universe, [0, 53, 61.5, 97.5])
    glucose['low'] = fuzz.trimf(glucose.universe, [53, 83.75, 125])
    glucose['medium'] = fuzz.trimf(glucose.universe, [70, 111.25, 162.5])
    glucose['high'] = fuzz.trimf(glucose.universe, [97.5, 143.75, 200])
    glucose['very high'] = fuzz.trapmf(glucose.universe, [125, 181.25, 200, 300])


def ageTrap(age: ctrl.Antecedent):
    age['young'] = fuzz.trapmf(age.universe, [0, 19, 21, 32.5])
    age['middle age'] = fuzz.trapmf(age.universe, [20, 27.25, 37.75, 47])
    age['old'] = fuzz.trapmf(age.universe, [32.5, 45, 49, 66.5])
    age['very old'] = fuzz.trapmf(age.universe, [47, 58.75, 100, 100])


def ageTri(age: ctrl.Antecedent):
    age['young'] = fuzz.trapmf(age.universe, [0, 18, 20, 32.5])
    age['middle age'] = fuzz.trimf(age.universe, [20, 32.5, 47])
    age['old'] = fuzz.trimf(age.universe, [32.5, 47, 66.5])
    age['very old'] = fuzz.trapmf(age.universe, [47, 66.5, 82, 100])


def bmiTrap(bmi: ctrl.Antecedent):
    bmi['underweight'] = fuzz.trapmf(bmi.universe, [0, 18.05, 18.45, 21.75])
    bmi['normal weight'] = fuzz.trapmf(bmi.universe, [18.25, 19.15, 24.35, 27.5])
    bmi['overweight'] = fuzz.trapmf(bmi.universe, [21.75, 25.5, 29.5, 32.5])
    bmi['obesity 1'] = fuzz.trapmf(bmi.universe, [27.5, 30.5, 34.5, 37.5])
    bmi['obesity 2'] = fuzz.trapmf(bmi.universe, [32.5, 35.5, 39.5, 42.5])
    bmi['obesity 3'] = fuzz.trapmf(bmi.universe, [37.5, 40.5, 80, 80])


def bmiTri(bmi: ctrl.Antecedent):
    bmi['underweight'] = fuzz.trapmf(bmi.universe, [0, 0, 18.25, 21.75])
    bmi['normal weight'] = fuzz.trimf(bmi.universe, [18.25, 21.75, 27.5])
    bmi['overweight'] = fuzz.trimf(bmi.universe, [21.75, 27.5, 32.5])
    bmi['obesity 1'] = fuzz.trimf(bmi.universe, [27.5, 32.5, 37.5])
    bmi['obesity 2'] = fuzz.trimf(bmi.universe, [32.5, 37.5, 42.5])
    bmi['obesity 3'] = fuzz.trapmf(bmi.universe, [37.5, 42.5, 80, 80])


def predictionTrap(prediction: ctrl.Antecedent):
    prediction['no'] = fuzz.trapmf(prediction.universe, [0, 0, 0.2, 0.8])
    prediction['yes'] = fuzz.trapmf(prediction.universe, [0.2, 0.8, 1, 1])


def predictionTri(prediction: ctrl.Antecedent):
    prediction['no'] = fuzz.trimf(prediction.universe, [0, 0, 0.8])
    prediction['yes'] = fuzz.trimf(prediction.universe, [0.2, 1, 1])


def buildRules(
    glucose: ctrl.Antecedent,
    age: ctrl.Antecedent,
    bmi: ctrl.Antecedent,
    prediction: ctrl.Antecedent,
):
    return [
        # glucose, age, no
        ctrl.Rule(glucose['very low'] &
                  (age['young'] | age['middle age'] | age['old']),
                  prediction['no']),
        ctrl.Rule(glucose['low'] &
                  (age['young'] | age['middle age']),
                  prediction['no']),
        ctrl.Rule(glucose['medium'] &
                  (age['young'] | age['middle age']),
                  prediction['no']),
        ctrl.Rule(glucose['high'] &
                  age['young'],
                  prediction['no']),

        # glucose, age, yes
        ctrl.Rule(glucose['high'] &
                  age['old'],
                  prediction['yes']),
        ctrl.Rule(glucose['very high'] &
                  (age['young'] | age['middle age'] | age['very old']),
                  prediction['yes']),

        # glucose, bmi, no
        ctrl.Rule(glucose['very low'] &
                  (bmi['normal weight'] | bmi['overweight'] | bmi['obesity 2']),
                  prediction['no']),
        ctrl.Rule(glucose['low'] &
                  (bmi['underweight'] | bmi['normal weight'] | bmi['overweight'] | bmi['obesity 1'] | bmi['obesity 2']),
                  prediction['no']),
        ctrl.Rule(glucose['medium'] &
                  (bmi['normal weight'] | bmi['overweight'] | bmi['obesity 2']),
                  prediction['no']),
        ctrl.Rule(glucose['high'] &
                  (bmi['normal weight']),
                  prediction['no']),

        # glucose, bmi, yes
        ctrl.Rule(glucose['very high'] &
                  (bmi['normal weight'] | bmi['overweight'] | bmi['obesity 1'] | bmi['obesity 2']),
                  prediction['yes']),
    ]


def buildRules2(
    glucose: ctrl.Antecedent,
    age: ctrl.Antecedent,
    bmi: ctrl.Antecedent,
    prediction: ctrl.Antecedent,
):
    return [
        # glucose, age, no
        ctrl.Rule(glucose['very low'] &
                  (age['young'] | age['middle age'] | age['old']),
                  prediction['no']),
        ctrl.Rule(glucose['low'] &
                  (age['young'] | age['middle age']),
                  prediction['no']),
        ctrl.Rule(glucose['medium'] &
                  (age['young'] | age['middle age']),
                  prediction['no']),
        ctrl.Rule(glucose['high'] &
                  age['young'],
                  prediction['no']),

        # glucose, age, yes
        ctrl.Rule(glucose['high'] &
                  age['old'],
                  prediction['yes']),
        ctrl.Rule(glucose['very high'] &
                  (age['young'] | age['middle age'] | age['very old']),
                  prediction['yes']),

        # glucose, bmi, no
        ctrl.Rule(glucose['very low'] &
                  (bmi['normal weight'] | bmi['overweight'] | bmi['obesity 2']),
                  prediction['no']),
        ctrl.Rule(glucose['low'] &
                  (bmi['underweight'] | bmi['normal weight'] | bmi['overweight'] | bmi['obesity 1'] | bmi['obesity 2'] | bmi['obesity 3']),
                  prediction['no']),
        ctrl.Rule(glucose['medium'] &
                  (bmi['normal weight'] | bmi['overweight'] | bmi['obesity 2'] | bmi['obesity 3']),
                  prediction['no']),
        ctrl.Rule(glucose['high'] &
                  (bmi['normal weight'] | bmi['overweight']),
                  prediction['no']),

        # glucose, bmi, yes
        ctrl.Rule(glucose['very high'] &
                  (bmi['normal weight'] | bmi['overweight'] | bmi['obesity 1'] | bmi['obesity 2'] | bmi['obesity 3']),
                  prediction['yes']),
    ]


In [4]:
glucose = ctrl.Antecedent(np.arange(56, 198+1, 0.01), 'glucose')
age = ctrl.Antecedent(np.arange(21, 81+1, 0.01), 'age')
bmi = ctrl.Antecedent(np.arange(18.2, 67.1+0.1, 0.01), 'bmi')
prediction = ctrl.Consequent(np.arange(0, 1+0.05, 0.05), 'prediction',
                             defuzzify_method="centroid")

glucoseTri(glucose=glucose)
ageTrap(age=age)
bmiTri(bmi=bmi)
predictionTri(prediction=prediction)

rules = buildRules(
    glucose=glucose,
    age=age,
    bmi=bmi,
    prediction=prediction,
)

prediction_ctrl = ctrl.ControlSystem(rules)
prediction_inference = ctrl.ControlSystemSimulation(prediction_ctrl)

prediction_inference.input['glucose'] = clean_data_df['Glucose'].to_numpy()
prediction_inference.input['age'] = clean_data_df['Age'].to_numpy()
prediction_inference.input['bmi'] = clean_data_df['BMI'].to_numpy()

prediction_inference.compute()
result_np = prediction_inference.output['prediction']

output_np = np.where(result_np > 0.50, 1, 0)
gt_np = clean_data_df['Outcome'].to_numpy()
total_sample = gt_np.shape[0]
accurate_sample = (output_np == gt_np).sum()
accuracy = (accurate_sample / total_sample) * 100
print(total_sample)
print(accurate_sample)
print(f"{accuracy:.2f}")

# glucose.view()
# age.view()
# bmi.view()
# prediction.view()


392
315
80.36


In [5]:
# prepare test run
rnd = np.random.default_rng(seed=4242)
selected_indices = rnd.integers(low=0, high=clean_data_df.shape[0], size=5).tolist()

prediction_ctrl = ctrl.ControlSystem(rules)
prediction_inference = ctrl.ControlSystemSimulation(prediction_ctrl)

test_run_result_dict = {
    "Glucose": [],
    "BMI": [],
    "Age": [],
    "Prediction": [],
    "Risk": [],
    "Ground Truth": [],
}

for ind in selected_indices:
    selected_series = clean_data_df.iloc[ind]
    glucose_input = selected_series['Glucose']
    bmi_input = selected_series['BMI']
    age_input = selected_series['Age']
    prediction_gt = selected_series['Outcome']

    prediction_inference.input['glucose'] = glucose_input
    prediction_inference.input['age'] = age_input
    prediction_inference.input['bmi'] = bmi_input

    prediction_inference.compute()
    prediction_confidence = prediction_inference.output['prediction']
    prediction_output = 1 if prediction_confidence > 0.5 else 0

    test_run_result_dict['Glucose'].append(glucose_input)
    test_run_result_dict['BMI'].append(bmi_input)
    test_run_result_dict['Age'].append(age_input)
    test_run_result_dict['Prediction'].append(prediction_output)
    test_run_result_dict['Ground Truth'].append(prediction_gt)
    test_run_result_dict['Risk'].append(prediction_confidence)

test_result_df = pd.DataFrame.from_dict(test_run_result_dict)

test_result_df


,Glucose,BMI,Age,Prediction,Risk,Ground Truth
0,105.0,34.9,25.0,0,0.286267,0.0
1,188.0,32.0,22.0,1,0.653529,1.0
2,123.0,34.1,28.0,0,0.278069,0.0
3,163.0,31.6,28.0,1,0.565641,1.0
4,142.0,24.7,21.0,0,0.377484,0.0
